<a href="https://colab.research.google.com/github/Daniel022de/DataEnginner_SoulCode/blob/main/Pipelines/pipelines_study_introduction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📗 **PIPELINE DE DADOS | CRIAÇÃO DE MODELOS**

Esse Notebook é referente as instruções feitas pelo professor **Igor Gondim** no bootcamp **engenharia de dados** da **SoulCode** na aula de introdução a automatização de processos através de ferramentas pipeline. A tecnologias escolhidas para consolidação de conhecimento foi o Apache Beam e o DataFlow Google Cloud.



! **Você pode encontrar esse notebook no meu repositório** [GitHub](https://github.com/Daniel022de/Bootcamp_SoulCode_EngenhariaDados)

! **Você pode entrar em contato comigo através do meu email** ddololiveira.pessoal@gmail.com **e** [Linkedin](https://www.linkedin.com/in/daniel-oliveira-503b0323b/).

! **Toda dúvida,recomendações e feedbacks serão bem-vindas.**



**Importante destacar que a automação de processos é feita quando necessária em situações corriqueiras**



*O que é uma pipeline?*

* Nada mais é que um fluxo de trabalho. Um processo com começo, meio e fim. Podendo ser manual (Usando Pandas/Pyspark por exemplo para processo de ETL) ou automatizada.

* Várias tecnologias criam pipelines.Exemplos:

**airflow** (muito usado | codificado e visual | Todo processo é executado na própria ferramenta), **apache beam** (Cria o modelo que pode ser usado em qualquer nuvem | A execução do modelo é feito em nuvem),**apache hop** ...

* O modelo não é eterno. São necessárias revisões, pois os datasets podem mudar, por exemplo, posições de colunas.


#▶ 1   Instalação

Realize cada uma em uma célula diferente e reiniciando ambiente a cada pip.

In [ ]:
# pip install --upgrade pip
# pip install apache_beam[interactive]
# pip install apache_beam[gcp]

In [ ]:
pip install --upgrade pip

In [ ]:
pip install apache_beam[interactive]

In [ ]:
pip install apache_beam[gcp]

#▶ 2    Importação

In [ ]:
import apache_beam as beam

#▶ 3    Pipelines

* **voos** - É o fluxo.

* **|** - Passo

* Os comandos precisam ter descrições

* **skip_header_lines = 1** - Pula a primeira lista que geralmente são os nomes das colunas, por padrão é 0. É necessário essa parte para que não haja erros em somatórias,divisões...

* **|'Imprimir o resultado'>> beam.Map(print)** - Faça somente no arquivo final ou para visualizar, pois o comando print descarta o arquivo da memória.

**Colunas:**


AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED'

##◼ 3.1 Imprimir arquivo

In [ ]:
p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/drive/MyDrive/DATASETS UPLOAD/voos.csv',skip_header_lines=1)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'Imprimir o resultado'>> beam.Map(print)
)

p1.run()

##◼ 3.2 Criar arquivo

In [ ]:
p1 = beam.Pipeline()

criar = (
      p1
      |'Extrair CSV' >> beam.io.ReadFromText('/content/drive/MyDrive/DATASETS UPLOAD/voos.csv',skip_header_lines=1)
      |'Separar os dados' >> beam.Map(lambda records: records.split(',')) 
      |'Criando arquivo' >> beam.io.WriteToText('arquivo.csv')
)

p1.run()

##◼ 3.3 Filtros

### ◼ 3.3.1 Origem

In [ ]:
p1 = beam.Pipeline()

filtro_origem = (
      p1
      |'Carregar CSV' >> beam.io.ReadFromText('/content/drive/MyDrive/DATASETS UPLOAD/voos.csv',skip_header_lines=1)
      |'Separador dos dados' >> beam.Map(lambda records: records.split(','))
      |'Filtro' >> beam.Filter(lambda records: records[1] == 'SFO')
      |'Imprimir' >> beam.Map(print)
)

p1.run()

### ◼ 3.3.2  Saida atrasada

In [ ]:
p1 = beam.Pipeline()

filtro_origem =(
      p1
      |'Carregar CSV' >> beam.io.ReadFromText('/content/drive/MyDrive/DATASETS UPLOAD/voos.csv',skip_header_lines=1)
      |'Separador dos dados' >> beam.Map(lambda records: records.split(','))
      |'Filtro' >> beam.Filter(lambda records: float(records[5]) > 0)
      |'Imprimindo' >> beam.Map(print)

      )
p1.run()


### ◼ 3.3.3  Voos atrasados

In [ ]:

p1 = beam.Pipeline() 

voos = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/drive/MyDrive/DATASETS UPLOAD/voos.csv',skip_header_lines=1)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'Filtro atraso'>> beam.Filter(lambda record: float(record[16]) > 0 )
    |'Imprimir o resultado'>> beam.Map(print)
  
)

p1.run()

##◼ 3.4  Agrupamento

In [ ]:
#Agrupamento minutos de atraso por aero

p1 = beam.Pipeline() 

qtd_minuto = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/drive/MyDrive/DATASETS UPLOAD/voos.csv',skip_header_lines=1)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'Filtro' >> beam.Filter(lambda record: float(record[16]) > 0)
    |'Agrupamento' >> beam.Map(lambda record: (record[1],float(record[16])))
    |'Construção da tabela atraso por minuto' >> beam.CombinePerKey(sum)
    |'Imprimir' >> beam.Map(print)
)
p1.run()

In [ ]:
#Agrupamento de ocorrência de atraso por aero
p1 = beam.Pipeline() 

qtd_ocorrencia = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/drive/MyDrive/DATASETS UPLOAD/voos.csv',skip_header_lines=1)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'Filtro' >> beam.Filter(lambda record: float(record[16]) > 0)
    |'Agrupamento' >> beam.Map(lambda record: (record[1],float(record[16])))
    |'Criar tabela quantidade ocorrencia' >> beam.combiners.Count.PerKey()
    |'Imprimir' >> beam.Map(print)
)

p1.run()


##◼ 3.5 coGroupBy

In [ ]:
# Criar nova tabela com dados agregados de duas pipelines

p1 = beam.Pipeline() 

qtd_minuto = (
    p1
    |'Extrair DataSet'>> beam.io.ReadFromText('/content/drive/MyDrive/DATASETS UPLOAD/voos.csv',skip_header_lines=1)
    |'Separação dos dados'>> beam.Map(lambda record: record.split(','))
    |'Filtro minutos' >> beam.Filter(lambda record: float(record[16]) > 0)
    |'Agrupamento dos dados' >> beam.Map(lambda record: (record[1],float(record[16])))
    |'Construção da tabela atraso por minuto' >> beam.CombinePerKey(sum)
)

qtd_ocorrencia = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/drive/MyDrive/DATASETS UPLOAD/voos.csv',skip_header_lines=1)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'Filtro' >> beam.Filter(lambda record: float(record[16]) > 0)
    |'Agrupamento' >> beam.Map(lambda record: (record[1],float(record[16])))
    |'Criar tabela quantidade ocorrencia' >> beam.combiners.Count.PerKey()
)

tabela = (
    {'qtd_ocorrencia':qtd_ocorrencia,'qtd_minuto':qtd_minuto}
    |'Unindo pernas' >> beam.CoGroupByKey()
    |'Print' >> beam.Map(print)
)


p1.run()

#▶ 6    Criação de model para GCP

In [ ]:
import apache_beam
import os
from apache_beam.options.pipeline_options import PipelineOptions


serviceAccount= '/content/drive/MyDrive/ACESSO A GCP/bc26-aulas-7791d7a8a1e0.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount


pipeline_options = {
    
    'project':'bc26-aulas',
    'runner':'DataflowRunner',
    'region':'southamerica-east1',
    'staging_location':'gs://ed7-datasets/staging/',
    'temp_location':'gs://ed7-datasets/temp/',
    'template_location':'gs://ed7-datasets/models/modelo_batch'
}




pipeline_options = PipelineOptions.from_dictionary(pipeline_options)

p1 = beam.Pipeline(options = pipeline_options)

qte_tempo = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('gs://ed7-datasets/brutos/voos.csv',skip_header_lines=1)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'Filtro atraso'>> beam.Filter(lambda record: float(record[16]) > 0 )
    |'Agregar as colunas' >> beam.Map(lambda record: (record[1],float(record[16])))
    |'Construir tabela por minuto somados' >> beam.CombinePerKey(sum)

)

qte_ocor = (
    p1
    |'Abrir do CSV'>> beam.io.ReadFromText('gs://ed7-datasets/brutos/voos.csv',skip_header_lines=1)
    |'Separar por virgula'>> beam.Map(lambda record: record.split(','))
    |'Filtro  aero atraso'>> beam.Filter(lambda record: float(record[16]) > 0 )
    |'Juntar as colunas' >> beam.Map(lambda record: (record[1],float(record[16])))
    |'Construir tabela por ocorrencia' >> beam.combiners.Count.PerKey()

)

tabela = (
    {'Num_de_Ocorrencias':qte_ocor,'Quant_minutos':qte_tempo}
    |'Agrupar as pernas' >> beam.CoGroupByKey()
    |'Load final' >> beam.io.WriteToText('gs://ed7-datasets/tratados/voosatr',file_name_suffix='.csv')


)

p1.run()